In [17]:
from IPython.core.display import HTML
HTML("""
<style>
code {
    padding:2px 4px !important;
    color: #c7254e !important;
    font-size: 90%;
    background-color: #f9f2f4 !important;
    border-radius: 4px !important;
    color: rgb(138, 109, 59);
    font-weight: bold;
}
mark {
    color: rgb(138, 109, 59) !important;
    font-weight: bold !important;
}
.container { width: 90% !important; }
table { font-size:15px !important; }
</style>
""")

In [18]:
import sys, math, os
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt    
import numpy as np
import statsmodels.formula.api as sm
import scipy.stats
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import operator

sys.path.insert(0, '../../src/data/')
import utils

savefig = False


%matplotlib inline
%load_ext autoreload  
%autoreload 2  

path_raw = "../../data/raw/beer_reviews"
trolls_file = '../../data/interim/trolls.csv'

assert os.path.isfile(os.path.join(path_raw, 'beer_reviews.csv')), "This data file doesn't exist yet, please run through 'make data'"
assert os.path.isfile(trolls_file), "The trolls file doesn't exist yet, please run through analysis 3.0 first!"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Introduction

The goal of this notebook is to try and answer the question
> Which of the factors (aroma, taste, appearance, palette) are most important in determining the overall quality of a beer?

Another way of wording the questions is:
> Which of the beer score factors (aroma, taste, appearance, palette) explain the most variance in the attribute `review_overall`.

**NOTE:** we learned several things in analysis [3.0_recommend_3_beers](3.0_recommend_3_beers.ipynb) that impact this analysis:
- we can remove the reviews that had no associated `review_profilename`
- there were numerous "troll" reviewers which should be removed from the dataset (trolls written to *../../data/interim/trolls.csv*)

In [19]:
# LOAD DATA
# we assume the file we're after is a
# single .csv in path_raw
for file in os.listdir(path_raw):
    file = os.path.join(path_raw, file)
    if os.path.isfile(file) and '.csv' in file: 
        dat_raw = pd.read_csv(file, encoding='utf-8') # NOTE: force utf-8 encoding because some beer_styles have accents in them
        
# this file only available if analysis 3.0 is run
trolls = pd.read_csv(trolls_file)

# create new copy of data
# 1. without reviews with missing profilename
# 2. without trolls
dat = dat_raw[(dat_raw.review_profilename.notnull()) & (~dat_raw.review_profilename.isin(trolls))].copy()

<hr>

## Univariate linear regression

In [20]:
factors = ['review_aroma','review_appearance','review_palate','review_taste']
all_scores_factors = factors + ['review_overall']
melt = pd.melt(dat, id_vars=['beer_beerid',
                      'review_overall'], value_vars=factors, var_name='factor', value_name='score')

dats = {}
for factor,df in melt.groupby('factor'):
    x = df.review_overall.tolist()
    y = df.score.tolist()
    
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    dats[factor] = [r_value**2, p_value, std_err]
#     print 'factor: %s, r^2: %.2f, p: %.4f, std err: %.4f' %(factor, r_value**2, p_value, std_err)

lm = pd.DataFrame(dats, index=['r^2','p','std err']).transpose()
lm.index.name = 'factor'
lm.to_csv('../../data/interim/beer_factors_linear_regression.csv')
lm

,r^2,p,std err
factor,,,
review_appearance,0.251726,0,0.000587
review_aroma,0.379439,0,0.000605
review_palate,0.492664,0,0.000535
review_taste,0.623780,0,0.000495


The factor `review_taste` is most highly correlated with `review_overall` having an r^2 of 0.62.

## Multivariate linear regression

In [15]:
y = 'review_overall'
x = "*".join(factors)
formula = '%s ~ %s' % (y, x)
reg_results = sm.ols(formula, data=dat[factors+[y]]).fit().summary()
with open('../../data/interim/ols.txt', 'w') as fout:
    fout.write(reg_results.as_text())
print(reg_results)

                            OLS Regression Results                            
Dep. Variable:         review_overall   R-squared:                       0.664
Model:                            OLS   Adj. R-squared:                  0.664
Method:                 Least Squares   F-statistic:                 2.087e+05
Date:                Tue, 20 Jun 2017   Prob (F-statistic):               0.00
Time:                        21:32:52   Log-Likelihood:            -8.6659e+05
No. Observations:             1586266   AIC:                         1.733e+06
Df Residuals:                 1586250   BIC:                         1.733e+06
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                                                                coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------

The multivariate model including all the beer factor scores has an r^2 value of 0.658, a value which isn't much higher than the model based on just using the `review_taste` as the only explanatory variable. This tells us that the 'review_taste' factor is the most important factor in predicting `review_overall`.

Let's try and train some different models to see if any other factors serve as good predictors.

In [21]:
# split our data into three sets
# training (60%)
# cv (20%)
# test (20%)
random = 42
dat_train, dat_cv, dat_test = utils.split_data(dat[all_scores_factors], random=random)

mdl = linear_model.LinearRegression(n_jobs=-1)
mdl.fit(dat_train[factors], dat_train['review_overall'])

print 'r^2 on test: %0.3f' %mdl.score(dat_test[factors], dat_test['review_overall'])

r^2 on test: 0.657


Values agree well with previous findings

## Random Forest regression

In [ ]:
# run a parameter grid search on the full dataset to find the optimized parameters
# for a random forest regression model. we use the full dataset becaues the
# GridSearchCV does a 3-fold cross-validation by default. Furthermore, we
# are only interested in training a RF model and extracting the
# feature importances out of it (instead of then predicting on a hold-out set)
param = {'n_estimators':[100,500], 'max_features':['auto',1/3.0], 'max_depth':[None,10,50]}
mdl = RandomForestRegressor(n_jobs=-1, random_state=random, oob_score=True)

clf = GridSearchCV(mdl, param, n_jobs=-1)
clf.fit(dat[factors], dat['review_overall'])
clf.best_params_

In [ ]:
mdl = RandomForestRegressor(n_jobs=-1, random_state=random, oob_score=True).set_params(**clf.best_params_)
mdl.fit(dat[factors], dat['review_overall'])
imp = {l :mdl.feature_importances_[i] for i,l in enumerate(factors)}
sorted(imp.items(), key=operator.itemgetter(1), reverse=True)

`review_taste` has the highest feature importance